In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = '2'

import sys
import argparse
import random
from pathlib import Path
import json

import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import pytorch_lightning as pl

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

from text_recognizer.data.base_data_module import BaseDataModule
from text_recognizer.data import IAMParagraphs
from text_recognizer.data.iam_paragraphs import resize_image, IMAGE_SCALE_FACTOR, get_transform
from text_recognizer.models import ResnetTransformer
from text_recognizer.lit_models import TransformerLitModel
from text_recognizer.paragraph_text_recognizer import ParagraphTextRecognizer
import text_recognizer.util as util

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
def _plot(image, figsize=(12,12), title='', vmin=0, vmax=255):
    plt.figure(figsize=figsize)
    if title:
        plt.title(title)
    plt.imshow(image, cmap='gray', vmin=vmin, vmax=vmax)

In [3]:
PARA_PROCESSED_DATA_DIRNAME = BaseDataModule.data_dirname() / "processed" / "iam_paragraphs"
PARA_PROCESSED_DATA_DIRNAME.exists()

True

In [8]:
text_recog = ParagraphTextRecognizer()

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:73] data. DefaultCPUAllocator: not enough memory: you tried to allocate 377487360 bytes. Buy new RAM!

### Prediction on trainval samples

In [6]:
trainval_para_filenames = list((PARA_PROCESSED_DATA_DIRNAME / 'trainval').glob('*.png'))
len(trainval_para_filenames)

1308

In [7]:
for _ in range(10):
    img_filename = random.choice(trainval_para_filenames)
    pred_str = text_recog.predict(img_filename)
    
    pil_img = util.read_image_pil(img_filename, grayscale=True)
    _plot(np.array(pil_img), title=pred_str)

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "C:\Users\ftmlab\Documents\hyoon\side_project\Full-stack-deeplearning\LABs\lab7 - 복사본\text_recognizer\models\resnet_transformer.py", line 188, in forward
            output = self.decode(x, y)  # (Sy, B, C) C: class(문자) 별 확률 
            output = torch.argmax(output, dim=-1)  # (Sy, B) 확률이 제일 큰 인덱스 추출
            output_tokens[:, Sy : Sy + 1] = output[-1:].unsqueeze(-1)  # Set the last output token
            ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
            
            # 배치 내의 모든 토큰이 end이거나 padding이면 루프 탈출
RuntimeError: output with shape [1, 1] doesn't match the broadcast shape [1, 1, 1]
